<a href="https://colab.research.google.com/github/PARTHIBAN-007/Transformers/blob/main/Mask_Filler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets evaluate transformers[sentencepiece] accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from transformers import AutoModelForMaskedLM

model_checkpoint = "distilbert-base-uncased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [3]:
distilbert_parameters = model.num_parameters()/1_000_000
print(f"Distilbert Parameters {round(distilbert_parameters)}M")
print("BERT Parameters : 110M")

Distilbert Parameters 67M
BERT Parameters : 110M


In [4]:
text = "This is a great [MASK]"
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
import torch

inputs = tokenizer(text,return_tensors="pt")
token_logits = model(**inputs).logits

mask_token_index = torch.where(inputs["input_ids"]==tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0,mask_token_index,:]
top_5_tokens = torch.topk(mask_token_logits,5,dim=1).indices[0].tolist()
for token in top_5_tokens:
  print(f"{text.replace(tokenizer.mask_token,tokenizer.decode([token]))}")

This is a great !
This is a great .
This is a great deal
This is a great adventure
This is a great ;


In [6]:
from datasets import load_dataset

imdb_dataset = load_dataset("imdb")
imdb_dataset

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [7]:
sample = imdb_dataset["train"].shuffle(seed=42).select(range(3))

for row in sample:
  print(f"Review : {row['text']}")
  print(f"Label : {row['label']}")

Review : There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier's plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it's the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...
Label : 1
Review : This movie is a great. The plot is very true to the book which is a classic written by Mark Twain. The movie starts of with a scene where Hank sings a song with a bunch of kids called "when you stub your toe on th

In [8]:
def tokenize_funxtion(examples):
  res = tokenizer(examples["text"])
  if tokenizer.is_fast: # Rust Language Tokenizer lIbrary
    res["word_ids"] = [res.word_ids(i) for i in range(len(res["input_ids"]))]
  return res

tokenized_dataset = imdb_dataset.map(
    tokenize_funxtion,
    batched =True,
    remove_columns = ["text","label"]
)
tokenized_dataset

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (720 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 50000
    })
})

In [9]:
tokenizer.model_max_length

512

In [10]:
chunk_size = 128

tokenized_samples = tokenized_dataset["train"][:3]

for idx,sample in enumerate(tokenized_samples["input_ids"]):
  print(f"Review : {idx} , length : {len(sample)}")

Review : 0 , length : 363
Review : 1 , length : 304
Review : 2 , length : 133


In [11]:
concatenated_samples = {
    k : sum(tokenized_samples[k],[]) for k in tokenized_samples.keys()
}
total_length = len(concatenated_samples["input_ids"])
print(f"Concatenated Review Length : {total_length}")


chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_samples.items()
}

for chunk in chunks["input_ids"]:
    print(f"Chunk length: {len(chunk)}'")

Concatenated Review Length : 800
Chunk length: 128'
Chunk length: 128'
Chunk length: 128'
Chunk length: 128'
Chunk length: 128'
Chunk length: 128'
Chunk length: 32'


In [12]:
def group_texts(examples):
  concatenated_examples = {k:sum(examples[k],[]) for k in examples.keys()}

  total_length = len(concatenated_examples[list(examples.keys())[0]])

  total_length = (total_length // chunk_size) * chunk_size

  res = {
      k : [t[i : i + chunk_size] for i in range(0,total_length,chunk_size)]
      for k,t in concatenated_examples.items()
  }

  res["labels"] = res["input_ids"].copy()
  return res

In [13]:
lm_dataset = tokenized_dataset.map(group_texts,batched=True)
lm_dataset

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 61291
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 59904
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 122957
    })
})

In [14]:
tokenizer.decode(lm_dataset["train"][1]["input_ids"])

"as the vietnam war and race issues in the united states. in between asking politicians and ordinary denizens of stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men. < br / > < br / > what kills me about i am curious - yellow is that 40 years ago, this was considered pornographic. really, the sex and nudity scenes are few and far between, even then it ' s not shot like some cheaply made porno. while my countrymen mind find it shocking, in reality sex and nudity are a major staple in swedish cinema. even ingmar bergman,"

In [17]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [18]:
samples = [lm_dataset["train"][i] for i in range(2)]

for sample in samples:
  sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
  print(f" {tokenizer.decode(chunk)}")

 [CLS] i rented i am curious - yellow from my video store because of all [MASK] controversy that surrounded it when it was first released [MASK] [MASK]. i also heard that [MASK] first it was seized by u. s. customs [MASK] it ever tried to enter this country, therefore being a fan of films considered " [MASK] [MASK] i really had to see this for myself. < br / [MASK] ss br / > the plot is centered around a young swedish [MASK] student named lena who wants [MASK] learn everything she can about life. in [MASK] she wants to focus her attentions to making [MASK] sort of documentary on what the average swede thought [MASK] certain [MASK] issues such
 [MASK] the vietnam war [MASK] [MASK] issues in the united states. in between [MASK] [MASK] and ordinary den [MASK]ns of stockholm aboutback opinions on politics, she has [MASK] with her drama teacher, [MASK], and married men. < br / > < br / > what kills me about i am curious - yellow is that 40 years [MASK], this [MASK] considered pornographic. 

In [19]:
import collections
import numpy as np

from transformers import default_data_collator

wwm_probability = 0.2

def whole_word_masking_data_collator(features):
  for feature in features:
    word_ids = feature.pop("word_ids")

  mapping = collections.defaultdict(list)
  current_word_idx = -1
  current_word =  None
  for idx, word_id in enumerate(word_ids):
    if word_id is not None:
      if word_id != current_word:
        current_word = word_id
        current_word_idx +=1
      mapping[current_word_idx].append(idx)

  mask = np.random.binomial(1, wwm_probability, (len(mapping),))
  input_ids = feature["input_ids"]
  labels = feature["labels"]
  new_labels = [-100] * len(labels)
  for word_id in np.where(mask)[0]:
    word_id = word_id.item()
    for idx in mapping[word_id]:
      new_labels[idx] = labels[idx]
      input_ids[idx] = tokenizer.mask_token_id

  feature["labels"] = new_labels

  return default_data_collator(features)

In [20]:
samples = [lm_dataset["train"][i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
  print(tokenizer.decode(chunk))

[CLS] i rented i am curious - yellow from my video store because of all the controversy that surrounded it when it was first released in 1967. i also heard that at first it was seized by u. s. customs if it ever tried to enter this country, therefore being a fan of films considered " controversial " i really had to see this for myself. < br / > < br / > the plot is centered around a young swedish drama student named lena who wants to learn everything she can about life. in particular she wants to focus her attentions to making some sort of documentary on what the average swede thought about certain political issues such
as the vietnam [MASK] and race issues in the united states. [MASK] between asking [MASK] [MASK] ordinary denizens of stockholm about their opinions on politics, she [MASK] sex with [MASK] [MASK] teacher, classmates, and married men. < [MASK] / > [MASK] [MASK] / > what kills me [MASK] i [MASK] [MASK] - yellow [MASK] that 40 years ago, this was considered pornographic [MA

In [43]:
train_size = 10_000
test_size = int(0.1*train_size)

downsampled_dataset = lm_dataset["train"].train_test_split(
    train_size=train_size,
    test_size=test_size,
    seed = 42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1000
    })
})

In [27]:
from huggingface_hub import notebook_login

notebook_login()

In [25]:
from transformers import TrainingArguments

batch_size = 64
logging_steps = len(downsampled_dataset["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

training_arguments = TrainingArguments(
    output_dir = f"{model_name}-FineTuned-imdb",
    overwrite_output_dir = True ,
    evaluation_strategy = "epoch",
    learning_rate = 1e-5,
    weight_decay = 0.01,
    per_device_train_batch_size = batch_size ,
    per_device_eval_batch_size = batch_size,
    push_to_hub = True,
    fp16 = True,
    logging_steps = logging_steps,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [28]:
from transformers import Trainer

trainer = Trainer(
    model = model,
    tokenizer = tokenizer,
    data_collator = data_collator ,
    args = training_arguments,
    train_dataset = downsampled_dataset["train"],
    eval_dataset = downsampled_dataset["test"]
)

<ipython-input-28-d12fbce5426f>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [30]:
import math
eval_res = trainer.evaluate()
print(f"perplexity : {math.exp(eval_res['eval_loss']):.2f}")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 2k23aids43 (unknown01-knowledge-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


perplexity : 21.94


In [31]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time
1,2.718200,2.550583,0.003000
2,2.647300,2.507732,0.003000
3,2.594200,2.536107,0.003000


TrainOutput(global_step=471, training_loss=2.6526973627175496, metrics={'train_runtime': 149.0539, 'train_samples_per_second': 201.269, 'train_steps_per_second': 3.16, 'total_flos': 994208670720000.0, 'train_loss': 2.6526973627175496, 'epoch': 3.0})

In [32]:
trainer.push_to_hub()

events.out.tfevents.1739275742.c1b9b08befc5.631.0:   0%|          | 0.00/7.27k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Parthiban007/distilbert-base-uncased-FineTuned-imdb/commit/febc28d3e2083641a57d3f032b06b53c15996b58', commit_message='End of training', commit_description='', oid='febc28d3e2083641a57d3f032b06b53c15996b58', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Parthiban007/distilbert-base-uncased-FineTuned-imdb', endpoint='https://huggingface.co', repo_type='model', repo_id='Parthiban007/distilbert-base-uncased-FineTuned-imdb'), pr_revision=None, pr_num=None)

In [37]:
def insert_random_mask(batch):
    features = [dict(zip(batch, t)) for t in zip(*batch.values())]
    masked_inputs = data_collator(features)
    return {"masked_" + k: v.numpy() for k, v in masked_inputs.items()}

In [ ]:
downsampled_dataset = downsampled_dataset.remove_columns(["word_ids"])

eval_dataset = downsampled_dataset["test"].map(
    insert_random_mask,
    batched=True,
    remove_columns=downsampled_dataset["test"].column_names,
)
eval_dataset = eval_dataset.rename_columns(
    {
        "masked_input_ids": "input_ids",
        "masked_attention_mask": "attention_mask",
        "masked_labels": "labels",
    }
)

In [45]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

batch_size = 64
train_dataloader = DataLoader(
    downsampled_dataset["train"],
    shuffle = True ,
    batch_size = batch_size ,
    collate_fn = data_collator

)

eval_dataloader = DataLoader(
    eval_dataset ,
    batch_size = batch_size,
    collate_fn = default_data_collator
)

In [66]:
from torch.optim import AdamW
from accelerate import Accelerator

optimizer = AdamW(model.parameters(),lr =0.01)
accelerator = Accelerator()

model , optimizer ,train_dataloader , eval_dataloaderr = accelerator.prepare(
    model,optimizer ,train_dataloader ,eval_dataloader
)


In [67]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear" ,
    optimizer = optimizer,
    num_warmup_steps = 0,
    num_training_steps = num_training_steps,
)

In [68]:
from huggingface_hub import get_full_repo_name

model_name =  "distilbert-base-uncased-FineTuned-imdb"
repo_name = get_full_repo_name(model_name)
repo_name

'Parthiban007/distilbert-base-uncased-FineTuned-imdb'

In [69]:
from tqdm.auto import tqdm
import torch
import math

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    model.train()
    for batch in train_dataloader:
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Move batch to GPU

        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        batch = {k: v.to(accelerator.device) for k, v in batch.items()}  # Move batch to GPU

        with torch.no_grad():
            outputs = model(**batch)

        loss = outputs.loss
        losses.append(accelerator.gather(loss.repeat(batch_size)))  # Ensure gathering happens correctly

    losses = torch.cat(losses)
    losses = losses[: len(eval_dataset)]

    try:
        perplexity = math.exp(torch.mean(losses))
    except OverflowError:
        perplexity = float("inf")

    print(f"Epoch {epoch}: Perplexity: {perplexity}")

    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)

    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        model.push_to_hub("Parthiban007/distilbert-base-uncased-FineTuned-imdb")
        tokenizer.push_to_hub("Parthiban007/distilbert-base-uncased-FineTuned-imdb")



  0%|          | 0/471 [00:00<?, ?it/s]

Epoch 0: Perplexity: 939.1157631369338


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


Epoch 1: Perplexity: 878.9126950925471


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


Epoch 2: Perplexity: 864.5341602367956


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


In [72]:
from transformers import pipeline

mask_filler = pipeline(
    "fill-mask",
    model = "Parthiban007/distilbert-base-uncased-FineTuned-imdb"
)

config.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


In [73]:
preds = mask_filler(text)

for pred in preds:
  print(pred['sequence'])

this is a great the
this is a great.
this is a great,
this is a great and
this is a great a
